https://www.api.gov.uk/ons/open-geography-portal/#open-geography-portal

# Libraries

In [1]:
import numpy as np
import pandas as pd

import json
import urllib.request

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

## Shape files from ONS

https://geoportal.statistics.gov.uk/

https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services

## Retrieve layer data

In [3]:
url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Clinical_Commissioning_Groups_April_2019_Boundaries_EN_BUC_2022/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'

region_url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Regions_December_2022_EN_BUC/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'

In [4]:
def create_geojson_url(dataset_name:str):
    url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/' + dataset_name + '/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'
    return url

In [5]:
def retrieve_geojson_from_url(url:str):
    with urllib.request.urlopen(url) as contents:
        geojson_data = json.loads(contents.read())
    
    shape_data = pd.json_normalize(geojson_data['features'])
    return shape_data

In [6]:
def save_geojson_from_url(url:str, filename:str):
    filename = filename + '.csv'
    shape_data = retrieve_geojson_from_url(url)
    shape_data.to_csv(filename, index=False)
    print(filename + ' successfully saved')

In [7]:
def save_geojson_from_dataset_name(dataset_name:str):
    url = create_geojson_url(dataset_name)
    save_geojson_from_url(url, dataset_name)

In [8]:
# with urllib.request.urlopen(url) as contents:
#     json_data = json.loads(contents.read())
# shape_data = pd.json_normalize(json_data['features'])

# shape_data.to_csv('test_shape_data.csv')

In [9]:
# with urllib.request.urlopen(region_url) as contents:
#     json_data = json.loads(contents.read())
# region_shape_data = pd.json_normalize(json_data['features'])

# region_shape_data.to_csv('test_region_shape_data.csv')

In [10]:
# with open('region/region_merged.geojson', 'r') as contents:
#     json_data = json.loads(contents.read())
# region_shape_data = pd.json_normalize(json_data['features'])
# region_shape_data.to_csv('region/region_merged_geojson_cleaned.csv')



## Shape directory

In [11]:
shape_directory_url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services?f=pjson'

with urllib.request.urlopen(shape_directory_url) as contents:
    shape_directory = json.loads(contents.read())
shape_directory = pd.json_normalize(shape_directory['services'])
shape_directory = shape_directory.loc[shape_directory['type']=='FeatureServer']

In [12]:
def shape_name_contains(str1:str, str2:str):
    str1 = str1.lower()
    str2 = str2.lower()
    return shape_directory[(shape_directory['name'].str.lower().str.contains(str1)) & (shape_directory['name'].str.lower().str.contains(str2))]

### icb

In [ ]:
icb_shape_directory = shape_name_contains('icb_jul_2022', 'buc')
icb_shape_directory

In [ ]:
# save_geojson_from_dataset_name(icb_shape_directory['name'].values[0])

### sub_icb

In [17]:
sub_icb_shape_directory = pd.concat([shape_name_contains('sub_integrated_care_board', 'bgc'),
                                     shape_name_contains('sub_integrated_care_board', 'buc'),
                                     shape_name_contains('sicbl', 'buc') # 2022
])

In [18]:
sub_icb_shape_directory

,name,type,url
1660,Sub_Integrated_Care_Board_Locations_April_2023_EN_BGC,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Sub_Integrated_Care_Board_Locations_April_2023_EN_BGC/FeatureServer
1616,SICBL_JUL_2022_EN_BUC,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/SICBL_JUL_2022_EN_BUC/FeatureServer


In [31]:
# save_geojson_from_dataset_name(sub_icb_shape_directory['name'].values[0])

SICBL_JUL_2022_EN_BUC.csv successfully saved


### ccg

In [ ]:
ccg_shape_directory = pd.concat([shape_name_contains('buc', 'clinical_commissioning_groups'), # 2019, 2021
                                 shape_name_contains('generalised', 'clinical_commissioning_groups'), # 2020
                                 shape_name_contains('ugcb', 'ccg') # 2015 to 2018
])

In [ ]:
ccg_shape_directory

In [ ]:
# for i in ccg_shape_directory['name']:
#     # print(i)
#     save_geojson_from_dataset_name(i)

### ltla

In [ ]:
ltla_shape_directory = pd.concat([shape_name_contains('local_authority_districts', 'uk_buc'), # 2019 to 2023
                                  shape_name_contains('lad', '2018_Boundaries_UK_BUC')
])

ltla_shape_directory

# shape_name_contains('local_authority_districts', 'generalised') # 2018
# shape_name_contains('local_authority_districts', 'ugcb') # 2019, 2020

In [ ]:
# for i in ltla_shape_directory['name']:
#     # print(i)
#     save_geojson_from_dataset_name(i)

### utla

In [ ]:
utla_shape_directory = pd.concat([shape_name_contains('counties_and_unitary_authorities', 'ew_buc'), # 2017, 2019
                                  shape_name_contains("counties_and_unitary_authorities_december_2018", 'gcb_ew'), # 2018
                                  shape_name_contains('counties_and_unitary_authorities', '2020_uk_bgc'), # 2020
                                  shape_name_contains('counties_and_unitary_authorities', 'en_buc'), # 2021
                                  shape_name_contains('Counties_and_Unitary_Authorities', '2011_gcb') # 2011
])

In [35]:
utla_shape_directory

,name,type,url
203,Counties_and_Unitary_Authorities_December_2011_GCB_EW_2022,FeatureServer,https://services1.arcgis.com/ESMARspQHYMw9BZ9/ArcGIS/rest/services/Counties_and_Unitary_Authorities_December_2011_GCB_EW_2022/FeatureServer


In [36]:
# for i in utla_shape_directory['name']:
#     # print(i)
#     save_geojson_from_dataset_name(i)

Counties_and_Unitary_Authorities_December_2011_GCB_EW_2022.csv successfully saved


### region

In [ ]:
region_shape_directory = pd.concat([shape_name_contains('Regions_December_2022_EN_BUC', '')
])

region_shape_directory

In [ ]:
# save_geojson_from_dataset_name(region_shape_directory['name'].values[0])

### STPs (pre Apr 2020) and STPs (from Apr 2020)

In [ ]:
stp_shape_directory = pd.concat([shape_name_contains('sustainability_and_transformation', 'buc'), # 2018, 2021
                                 shape_name_contains('stp', '_gcb') # 2017, 2019, 2020
])

In [ ]:
stp_shape_directory

In [ ]:
# for i in stp_shape_directory['name']:
#     save_geojson_from_dataset_name(i)

# append files together

In [19]:
gsheet_areas = pd.read_csv('https://docs.google.com/spreadsheets/u/0/d/1yWPMu8cHllEdVl0kuUjjJR5V8yVrnU4jZTIlM1a1XGg/gviz/tq?tqx=out:csv&tq&gid=963757659&headers=1')
files_required = gsheet_areas.loc[:, ['shape_file']].dropna()

In [20]:
files_required

,shape_file
0,ICB_JUL_2022_EN_BUC_V3.csv
1,Sustainability_and_Transformation_Partnerships_April_2021_EN_BUC_V2_2022.csv
2,STP_Apr_2019_GCB_in_England_2022.csv
3,Regions_December_2022_EN_BUC.csv
4,SICBL_JUL_2022_EN_BUC.csv
5,Clinical_Commissioning_Groups_April_2021_EN_BUC_2022.csv
6,Clinical_Commissioning_Groups_April_2020_Ultra_Generalised_Boundaries_EN_2022.csv
7,Clinical_Commissioning_Groups_April_2019_Boundaries_EN_BUC_2022.csv
8,CCG_Apr_2018_UGCB_in_England_2022.csv
9,CCG_Apr_2017_UGCB_in_England_V4_2022.csv


In [21]:
area_id_to_shape_file = gsheet_areas.loc[:, ['AreaTypeId', 'shape_file']].dropna()

In [23]:
def code_and_name_fields(folder_name):
    final_list=[]
    
    for i in files_required.values:
        filename = i[0]
        filepath = folder_name + '/' + filename
        data = pd.read_csv(filepath)
        
        to_append_list = [filename, data.filter(regex='CD|cd').columns[0], data.filter(regex='NM|nm').columns[0]]
        
        final_list += [to_append_list]
        final_df = pd.DataFrame(final_list, columns=['shape_file', 'code_field', 'name_field'])
        
    return final_df


In [25]:
data = code_and_name_fields('shapes')
# data.to_csv('2023-08-10_shape_code_and_name_fields.csv', index=False)
data.head()

,shape_file,code_field,name_field
0,ICB_JUL_2022_EN_BUC_V3.csv,properties.ICB22CD,properties.ICB22NM
1,Sustainability_and_Transformation_Partnerships_April_2021_EN_BUC_V2_2022.csv,properties.STP21CD,properties.STP21NM
2,STP_Apr_2019_GCB_in_England_2022.csv,properties.stp19cd,properties.stp19nm
3,Regions_December_2022_EN_BUC.csv,properties.RGN22CD,properties.RGN22NM
4,SICBL_JUL_2022_EN_BUC.csv,properties.SICBL22CD,properties.SICBL22NM


In [26]:
def append_shapes(folder_name):
    shape_fields = pd.read_csv('2023-08-10_shape_code_and_name_fields.csv')
    
    appended_shapes = pd.DataFrame()
    
    for i, j in enumerate(files_required.values):
        filename = j[0]
        filepath = folder_name + '/' + filename
        shapes = pd.read_csv(filepath)
        
        rename_cols = {'code_field': 'area_code',
                       'name_field': 'area_name'}
        
        column_mapper_reversed = shape_fields.loc[shape_fields['shape_file']==filename, ['code_field', 'name_field']].rename(rename_cols, axis=1).loc[i].to_dict()
        column_mapper = {value: key for key, value, in column_mapper_reversed.items()}
        
        shapes = shapes.rename(column_mapper, axis=1)
        shapes['area_type_id'] = area_id_to_shape_file.loc[area_id_to_shape_file['shape_file']==filename, ['AreaTypeId']].values[0][0]
        shapes['area_code'] = shapes['area_code'] + '_' + shapes['area_type_id'].astype(str)
              
        appended_shapes = pd.concat([appended_shapes, shapes])
        
    return appended_shapes


In [27]:
append_shapes('shapes').to_csv('shapes/2023-08-10_shapes_appended.csv', index=False)

# do some rough cleaning to existing file

In [28]:
import pandas as pd

shape = pd.read_csv('shapes/2023-08-10_shapes_appended.csv')
cols = ['geometry.coordinates', 'area_code', 'area_type_id']
cols_dict = {'geometry.coordinates': 'shape'}

shape = shape.loc[:,cols].rename(columns=cols_dict)

# shape.to_csv('2023-08-10_shapes.csv', index=False)

In [29]:
shape.loc[shape.loc[:,'area_type_id']==66].to_csv('66_test.csv', index=False)